In [1]:
# Import necessary libraries.
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
import librosa
import librosa.display

In [2]:
import requests

fname = "music.zip"
url = "https://osf.io/drjhb/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [3]:
from zipfile import ZipFile

with ZipFile(fname, 'r') as zipObj:
  # Extract all the contents of zip file in different directory
  zipObj.extractall()

In [4]:
import pandas as pd
import shutil

In [6]:
sample_path = 'Data/genres_original/jazz/jazz.00000.wav'
y, sample_rate = librosa.load(sample_path)
y_resamble= librosa.resample(y, 22050, 11025)
librosa.feature.chroma_stft(y_resamble ,11025  ) .shape

(12, 647)

In [7]:
dir= 'Data/genres_original/' # the dir with all the genres
new_dir= 'new_music' # new dir for us
try:
    shutil.rmtree(new_dir, ignore_errors = False)
except:
    print('no file continue')
os.mkdir(new_dir)
genre_dict={}
for genre in os.listdir( dir ):
    genre_dir= dir+genre 
  
    os.mkdir(f'{new_dir}/{genre}')

    genre_dict[genre]= []
    print(f'we are in {genre} directory')
    use_this_dir= f'{new_dir}/{genre}' # move files here
    for music_file in os.listdir(genre_dir): # this is the music file name
        music_loc =os.path.join( genre_dir ,music_file) # the address for the music file
        
        try:
            y, sr = librosa.load(music_loc)
            y_resamble= librosa.resample(y, 22050, 11025) # using subsampling
            S_C= librosa.feature.spectral_centroid( y_resamble, sr=11025  )
            S_RO= librosa.feature.spectral_rolloff( y_resamble, sr=11025  )
            C_STFT= librosa.feature.chroma_stft( y_resamble, sr=11025  )
            S = librosa.feature.melspectrogram( y_resamble, sr=11025  )
            S_DB = librosa.amplitude_to_db(S, ref=np.max)

            genre_dict[genre].append( [S_DB, S_C , S_RO , C_STFT ] )  # tries to add the signal to the genre

        except:
            print('error')

we are in jazz directory


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


error
we are in pop directory
we are in country directory
we are in rock directory
we are in blues directory
we are in hiphop directory
we are in classical directory
we are in metal directory
we are in disco directory
we are in reggae directory


In [8]:
# taking only 99 signals instead of 100 because we have 1 faulty file inside genre: jazz
for arr in genre_dict.keys():
    if arr == 'jazz':
        pass
    else:
        del genre_dict[arr][-1]

In [9]:
df= pd.DataFrame( genre_dict)# , dtype= np.float32 )
df.head()

,jazz,pop,country,rock,blues,hiphop,classical,metal,disco,reggae
0,"[[[-65.298096, -70.04863, -77.817276, -80.0, -...","[[[-71.087036, -75.05148, -80.0, -80.0, -80.0,...","[[[-46.835487, -58.266075, -80.0, -80.0, -67.5...","[[[-71.120415, -78.295044, -80.0, -73.21907, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-37.62037, -46.842506, -59.62098, -45.18292...","[[[-80.0, -80.0, -80.0, -76.15962, -69.93909, ...","[[[-41.019203, -52.859505, -80.0, -80.0, -80.0...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -..."
1,"[[[-64.74619, -74.24568, -80.0, -80.0, -80.0, ...","[[[-77.5818, -80.0, -80.0, -80.0, -80.0, -80.0...","[[[-55.32198, -68.65732, -80.0, -80.0, -80.0, ...","[[[-55.925564, -66.73613, -80.0, -80.0, -80.0,...","[[[-55.552357, -67.591354, -80.0, -80.0, -80.0...","[[[-39.519073, -51.55992, -80.0, -80.0, -80.0,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-46.913574, -47.10653, -35.61605, -41.73833...","[[[-33.52887, -45.750977, -72.40613, -69.67498...","[[[-80.0, -79.159966, -77.46018, -80.0, -80.0,..."
2,"[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -74.938545, -63.9778, -59.64665, -59...","[[[-72.3529, -79.62429, -80.0, -72.23688, -71....","[[[-44.20174, -52.632538, -62.276447, -61.7302...","[[[-47.816505, -55.820377, -64.73056, -60.7971...","[[[-65.22782, -75.527054, -80.0, -80.0, -80.0,...","[[[-58.97224, -59.695835, -61.236443, -59.2669...","[[[-45.245186, -47.934055, -41.114285, -49.475...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -73.41835, -68...."
3,"[[[-67.64529, -60.859486, -56.007336, -54.7370...","[[[-65.58223, -70.38751, -70.984955, -80.0, -8...","[[[-64.84816, -56.42335, -48.77673, -50.60338,...","[[[-52.912495, -52.859367, -60.448555, -73.025...","[[[-64.39882, -62.83524, -58.741096, -58.94006...","[[[-60.656006, -67.13735, -70.674904, -61.2579...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-77.368744, -79.86925, -80.0, -80.0, -80.0,...","[[[-51.39866, -51.435326, -51.838654, -61.5722...","[[[-53.92138, -64.062325, -80.0, -80.0, -80.0,..."
4,"[[[-56.985027, -68.57016, -64.7605, -55.85546,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-57.02016, -68.89549, -80.0, -80.0, -58.502...","[[[-60.12925, -57.828476, -55.82229, -47.68387...","[[[-47.42361, -55.557312, -68.46201, -72.0973,...","[[[-57.81318, -64.48157, -80.0, -80.0, -80.0, ...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-48.585335, -60.65073, -80.0, -80.0, -80.0,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -..."


In [22]:
print( df.jazz[0][0].shape , 'for S_DB' )
print(df.jazz[0][1].shape  , 'for S_C' )
print(df.jazz[0][2].shape , ' for S_RO ')
print(df.jazz[0][3].shape , 'for C_STFT ')

(128, 647) for S_DB
(1, 647) for S_C
(1, 647)  for S_RO 
(12, 647) for C_STFT 


In [14]:
df_as_array= df.to_numpy()

In [25]:
# first dim for class , sec for audio file , third for feature
print( df_as_array[0][0][0].shape , 'for S_DB' )
print(df_as_array[0][0][1].shape  , 'for S_C' )
print(df_as_array[0][0][2].shape , ' for S_RO ')
print(df_as_array[0][0][3].shape , 'for C_STFT ')

(128, 647) for S_DB
(1, 647) for S_C
(1, 647)  for S_RO 
(12, 647) for C_STFT 


In [27]:
np.save('4_features_array.npy' , df_as_array  )

# for loading the data

In [28]:
data_tmp = np.load('/content/4_features_array.npy' , allow_pickle=True)

In [29]:
data= pd.DataFrame(data_tmp)

In [82]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,"[[[-65.298096, -70.04863, -77.817276, -80.0, -...","[[[-71.087036, -75.05148, -80.0, -80.0, -80.0,...","[[[-46.835487, -58.266075, -80.0, -80.0, -67.5...","[[[-71.120415, -78.295044, -80.0, -73.21907, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-37.62037, -46.842506, -59.62098, -45.18292...","[[[-80.0, -80.0, -80.0, -76.15962, -69.93909, ...","[[[-41.019203, -52.859505, -80.0, -80.0, -80.0...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -..."
1,"[[[-64.74619, -74.24568, -80.0, -80.0, -80.0, ...","[[[-77.5818, -80.0, -80.0, -80.0, -80.0, -80.0...","[[[-55.32198, -68.65732, -80.0, -80.0, -80.0, ...","[[[-55.925564, -66.73613, -80.0, -80.0, -80.0,...","[[[-55.552357, -67.591354, -80.0, -80.0, -80.0...","[[[-39.519073, -51.55992, -80.0, -80.0, -80.0,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-46.913574, -47.10653, -35.61605, -41.73833...","[[[-33.52887, -45.750977, -72.40613, -69.67498...","[[[-80.0, -79.159966, -77.46018, -80.0, -80.0,..."
2,"[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -74.938545, -63.9778, -59.64665, -59...","[[[-72.3529, -79.62429, -80.0, -72.23688, -71....","[[[-44.20174, -52.632538, -62.276447, -61.7302...","[[[-47.816505, -55.820377, -64.73056, -60.7971...","[[[-65.22782, -75.527054, -80.0, -80.0, -80.0,...","[[[-58.97224, -59.695835, -61.236443, -59.2669...","[[[-45.245186, -47.934055, -41.114285, -49.475...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -73.41835, -68...."
3,"[[[-67.64529, -60.859486, -56.007336, -54.7370...","[[[-65.58223, -70.38751, -70.984955, -80.0, -8...","[[[-64.84816, -56.42335, -48.77673, -50.60338,...","[[[-52.912495, -52.859367, -60.448555, -73.025...","[[[-64.39882, -62.83524, -58.741096, -58.94006...","[[[-60.656006, -67.13735, -70.674904, -61.2579...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-77.368744, -79.86925, -80.0, -80.0, -80.0,...","[[[-51.39866, -51.435326, -51.838654, -61.5722...","[[[-53.92138, -64.062325, -80.0, -80.0, -80.0,..."
4,"[[[-56.985027, -68.57016, -64.7605, -55.85546,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-57.02016, -68.89549, -80.0, -80.0, -58.502...","[[[-60.12925, -57.828476, -55.82229, -47.68387...","[[[-47.42361, -55.557312, -68.46201, -72.0973,...","[[[-57.81318, -64.48157, -80.0, -80.0, -80.0, ...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -...","[[[-48.585335, -60.65073, -80.0, -80.0, -80.0,...","[[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -..."


In [83]:
# creating X for the signals
# y for the labels
y=[]
S_DB=[]
S_C= []
S_RO= []
C_STFT=[]
for label in data:
    print(label)
    for sig in data[label]:
        y.append(label)

        S_DB.append(sig[0])
        S_C.append(sig[1])
        S_RO.append(sig[2])
        C_STFT.append(sig[3])


0
1
2
3
4
5
6
7
8
9


In [36]:
data[0][0][1].shape

(1, 647)

In [65]:
tmp_arr= np.array(df_as_array)
for i in range(data.shape[0]): # files
    for m in range(10): #  labels
        for f in range(4): # features
            tmp_arr[i][m][f]=data.iloc[i][m][f][:,:645]

In [66]:
# first dim for class , sec for audio file , third for feature
print( tmp_arr[0][0][0].shape , 'for S_DB' )
print(tmp_arr[0][2][1].shape  , 'for S_C' )
print(tmp_arr[0][0][2].shape , ' for S_RO ')
print(tmp_arr[0][3][3].shape , 'for C_STFT ')

(128, 645) for S_DB
(1, 645) for S_C
(1, 645)  for S_RO 
(12, 645) for C_STFT 


In [67]:
X= np.squeeze( tmp_arr.copy() )

In [84]:
y= np.array( y )

In [85]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,

## saving the features and labels

In [87]:
np.save( 'all_the_features.npy',X)
np.save('all_the_labels.npy' ,y)

# loading the data

In [92]:
x_load= np.load('all_the_features.npy', allow_pickle=True)
y_load= np.load('all_the_labels.npy', allow_pickle=True)

In [99]:
x_load[0][0][0].shape

(128, 645)

In [106]:
# # from  sklearn.model_selection import train_test_split 
# # x_train , x_test , y_train , y_test = train_test_split( x_load[:][:][:][0] , y_load , train_size=0.8)

# # changing them into torch format
# x_train = torch.tensor(x_load[:,0] )
# ##x_test = torch.tensor(x_test)

# y_train =torch.tensor(y_load).type(torch.IntTensor) 
# #y_test = torch.tensor(y_test).type(torch.IntTensor)

# train = torch.utils.data.TensorDataset(x_train,y_train)
# #test = torch.utils.data.TensorDataset(x_test,y_test)

# copying 

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [107]:
!cp  '/content/all_the_features.npy' '/content/drive/MyDrive/NMA_DL_project'

In [108]:
!cp  '/content/all_the_labels.npy' '/content/drive/MyDrive/NMA_DL_project'